In [52]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np 
#from causenet.datastream import DataStream, prepare_datastream
from sklearn.preprocessing import OneHotEncoder

In [53]:
X_mnist = None
y_mnist = None

In [54]:
def monte_carlo_error(g_hat, data_fn, ntest=5000, has_latent=False, debug=False):
    seed = np.random.randint(1e9)
    try:
        # test = True ensures we draw test set images
        x, z, t, y, g_true = data_fn(ntest, seed, test=True)
    except ValueError:
        warnings.warn("Too few images, reducing test set size")
        ntest = int(ntest * 0.7)
        # test = True ensures we draw test set images
        x, z, t, y, g_true = data_fn(ntest, seed, test=True)

    ## re-draw to get new independent treatment and implied response
    t = np.linspace(np.percentile(t, 2.5),np.percentile(t, 97.5),ntest).reshape(-1, 1)
    ## we need to make sure z _never_ does anything in these g functions (fitted and true)
    ## above is necesary so that reduced form doesn't win
    if has_latent:
        x_latent, _, _, _, _ = data_fn(ntest, seed, images=False)
        y = g_true(x_latent, z, t)
    else:
        y = g_true(x, z, t)
    y_true = y.flatten()
    y_hat = g_hat(x, z, t).flatten()
    return ((y_hat - y_true)**2).mean()

In [55]:
def loadmnist():
    '''
    Load the mnist data once into global variables X_mnist and y_mnist.
    '''
    from keras.datasets import mnist
    global X_mnist
    global y_mnist
    train, test = mnist.load_data()
    X_mnist = []
    y_mnist = []
    for d in [train, test]:
        X, y = d
        X = X.astype('float32')
        X /= 255.
        idx = np.argsort(y)
        X_mnist.append(X[idx, :, :])
        y_mnist.append(y[idx])

In [56]:
def get_images(digit, n, seed=None, testset=False):
    if X_mnist is None:
        loadmnist()
    is_test = int(testset)
    rng = np.random.RandomState(seed)
    X_i = X_mnist[is_test][y_mnist[is_test] == digit, :, :]
    n_i, i, j = X_i.shape
    perm = rng.permutation(np.arange(n_i))
    if n > n_i:
        raise ValueError('You requested %d images of digit %d when there are \
                          only %d unique images in the %s set.' % (n, digit, n_i, 'test' if testset else 'training'))
    return X_i[perm[0:n], :, :].reshape((n,i*j))

In [57]:
def one_hot(col, **kwargs):
    z = col.reshape(-1,1)
    enc = OneHotEncoder(sparse=False, **kwargs)
    return enc.fit_transform(z)

In [58]:
def get_test_valid_train(generator, n, batch_size=128, seed=123, **kwargs):
    x, z, t, y, g = generator(n=int(n*0.6), seed=seed, **kwargs)
    train = prepare_datastream(x, z, t, y, True, batch_size, **kwargs)
    x, z, t, y, g = generator(n=int(n*0.2), seed=seed+1, **kwargs)
    valid = prepare_datastream(x, z, t, y, False, batch_size, **kwargs)
    x, z, t, y, g = generator(n=int(n*0.2), seed=seed+2, **kwargs)
    test = prepare_datastream(x, z, t, y, False, batch_size, **kwargs)
    return train, valid, test, g

---

## 4. Experiments

### 4.1 Simulations

$$\psi_t = 2\{ \frac{(t-5)^4}{600} + exp[-4(t-5)^2] + \frac{t}{10} -2\}$$

In [59]:
def sensf(x):
    return 2.0*((x - 5)**4 / 600 + np.exp(-((x - 5)/0.5)**2) + x/10. - 2)

def emocoef(emo):
    emoc = (emo * np.array([1., 2., 3., 4., 5., 6., 7.])[None, :]).sum(axis=1)
    return emoc

---

In [60]:
psd = 3.7
pmu = 17.779
ysd = 158.#292.
ymu = -292.1

In [61]:
def storeg(x, price):
    emoc = emocoef(x[:, 1:])
    time = x[:, 0]
    g = sensf(time)*emoc*10. + (emoc*sensf(time)-2.0)*(psd*price.flatten() + pmu)
    y = (g - ymu)/ysd
    return y.reshape(-1, 1)

In [105]:

def demand(n, seed=1, ynoise=1., pnoise=1., ypcor=0.8, use_images=False, test=False):
    
    rng = np.random.RandomState(seed)

    # covariates: time and emotion
    time = rng.rand(n) * 10
    emotion_id = rng.randint(0, 7, size=n)
    emotion = one_hot(emotion_id, n_values=7)
    if use_images:
        idx = np.argsort(emotion_id)
        emotion_feature = np.zeros((0, 28*28))
        for i in range(7):
            img = get_images(i, np.sum(emotion_id == i), seed, test)
            emotion_feature = np.vstack([emotion_feature, img])
        reorder = np.argsort(idx)
        emotion_feature = emotion_feature[reorder, :]
    else:
        emotion_feature = emotion

    # random instrument
    z = rng.randn(n)

    # z -> price
    v = rng.randn(n)*pnoise
    price = sensf(time)*(z + 3)  + 25.
    price = price + v
    price = (price - pmu)/psd

    # true observable demand function
    x = np.concatenate([time.reshape((-1, 1)), emotion_feature], axis=1)
    x_latent = np.concatenate([time.reshape((-1, 1)), emotion], axis=1)
    g = lambda x, z, p: storeg(x, p) # doesn't use z

    # errors 
    e = (ypcor*ynoise/pnoise)*v + rng.randn(n)*ynoise*np.sqrt(1-ypcor**2)
    e = e.reshape(-1, 1)
    
    # response
    y = g(x_latent, None, price) + e

    return (x,
            z.reshape((-1, 1)),
            price.reshape((-1, 1)),
            y.reshape((-1, 1)),
            g)

In [106]:
seed = 1
rng = np.random.RandomState(seed)

In [107]:
n = 20
time = rng.rand(n) * 10 # time ~ unif(0,10)
time

array([  4.17022005e+00,   7.20324493e+00,   1.14374817e-03,
         3.02332573e+00,   1.46755891e+00,   9.23385948e-01,
         1.86260211e+00,   3.45560727e+00,   3.96767474e+00,
         5.38816734e+00,   4.19194514e+00,   6.85219500e+00,
         2.04452250e+00,   8.78117436e+00,   2.73875932e-01,
         6.70467510e+00,   4.17304802e+00,   5.58689828e+00,
         1.40386939e+00,   1.98101489e+00])

In [108]:
emotion_id = rng.randint(0, 7, size=n)
emotion_id # s

array([5, 1, 1, 5, 6, 1, 1, 0, 4, 1, 0, 0, 5, 3, 2, 1, 0, 6, 3, 6])

In [109]:
emotion = one_hot(emotion_id, n_values=7)
emotion

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.]])

##### not using mnist

In [110]:
emotion_feature = emotion

In [111]:
z = rng.randn(n) # z ~ n(0,1)
z

array([ 0.04221375,  0.58281521, -1.10061918,  1.14472371,  0.90159072,
        0.50249434,  0.90085595, -0.68372786, -0.12289023, -0.93576943,
       -0.26788808,  0.53035547, -0.69166075, -0.39675353, -0.6871727 ,
       -0.84520564, -0.67124613, -0.0126646 , -1.11731035,  0.2344157 ])

In [112]:
# calculate by hand
psd = 3.7
pmu = 17.779
ysd = 158.#292.
ymu = -292.1

In [113]:
pnoise = 1 # by default is 1, 
v = rng.randn(n)*pnoise # v ~ n(0,1)

price = sensf(time)*(z + 3)  + 25. + v

# price = (price - pmu)/psd # why? 

price

array([ 17.42045266,  16.853782  ,  21.16449957,  10.25056606,
        11.81661901,  16.55375011,  12.16037729,  16.74299786,
        16.05749248,  23.32762136,  16.88724904,  16.47242397,
        17.59777245,  20.58046099,  18.57915167,  18.98161047,
        17.72544476,  18.48286276,  19.88637564,  15.17053345])

In [114]:
time.reshape((-1,1))

array([[  4.17022005e+00],
       [  7.20324493e+00],
       [  1.14374817e-03],
       [  3.02332573e+00],
       [  1.46755891e+00],
       [  9.23385948e-01],
       [  1.86260211e+00],
       [  3.45560727e+00],
       [  3.96767474e+00],
       [  5.38816734e+00],
       [  4.19194514e+00],
       [  6.85219500e+00],
       [  2.04452250e+00],
       [  8.78117436e+00],
       [  2.73875932e-01],
       [  6.70467510e+00],
       [  4.17304802e+00],
       [  5.58689828e+00],
       [  1.40386939e+00],
       [  1.98101489e+00]])

In [115]:
emotion_feature

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [116]:
np.concatenate([time.reshape((-1, 1)), emotion_feature], axis=1)

array([[  4.17022005e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00,   0.00000000e+00],
       [  7.20324493e+00,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00],
       [  1.14374817e-03,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00],
       [  3.02332573e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00,   0.00000000e+00],
       [  1.46755891e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   1.00000000e+00],
       [  9.23385948e-01,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.0

In [117]:
np.concatenate([time.reshape((-1, 1)), emotion], axis=1)

array([[  4.17022005e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00,   0.00000000e+00],
       [  7.20324493e+00,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00],
       [  1.14374817e-03,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00],
       [  3.02332573e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00,   0.00000000e+00],
       [  1.46755891e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   1.00000000e+00],
       [  9.23385948e-01,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.0

In [118]:
# true observable demand function
x = np.concatenate([time.reshape((-1, 1)), emotion_feature], axis=1)
x_latent = np.concatenate([time.reshape((-1, 1)), emotion], axis=1)
g = lambda x, z, p: storeg(x, p) # doesn't use z


In [119]:
emotion = x[:, 1:]

emoc = emocoef(emotion)
emoc

array([ 6.,  2.,  2.,  6.,  7.,  2.,  2.,  1.,  5.,  2.,  1.,  1.,  6.,
        4.,  3.,  2.,  1.,  7.,  4.,  7.])

In [120]:
emotion_id # s

array([5, 1, 1, 5, 6, 1, 1, 0, 4, 1, 0, 0, 5, 3, 2, 1, 0, 6, 3, 6])

In [121]:
time = x[:, 0]
time # t

array([  4.17022005e+00,   7.20324493e+00,   1.14374817e-03,
         3.02332573e+00,   1.46755891e+00,   9.23385948e-01,
         1.86260211e+00,   3.45560727e+00,   3.96767474e+00,
         5.38816734e+00,   4.19194514e+00,   6.85219500e+00,
         2.04452250e+00,   8.78117436e+00,   2.73875932e-01,
         6.70467510e+00,   4.17304802e+00,   5.58689828e+00,
         1.40386939e+00,   1.98101489e+00])

In [122]:
def sensf(t):
    return 2.0*((t - 5)**4 / 600 + np.exp(-((t - 5)/0.5)**2) + t/10. - 2)

$$ g = 10 \times s \phi_t  + p \times s \phi_t - 2p + e $$

$$ g = 10 \times s \phi_t  + (s \phi_t - 2) \times p$$

In [123]:
price

array([ 17.42045266,  16.853782  ,  21.16449957,  10.25056606,
        11.81661901,  16.55375011,  12.16037729,  16.74299786,
        16.05749248,  23.32762136,  16.88724904,  16.47242397,
        17.59777245,  20.58046099,  18.57915167,  18.98161047,
        17.72544476,  18.48286276,  19.88637564,  15.17053345])

In [124]:
price.flatten()

array([ 17.42045266,  16.853782  ,  21.16449957,  10.25056606,
        11.81661901,  16.55375011,  12.16037729,  16.74299786,
        16.05749248,  23.32762136,  16.88724904,  16.47242397,
        17.59777245,  20.58046099,  18.57915167,  18.98161047,
        17.72544476,  18.48286276,  19.88637564,  15.17053345])

In [125]:
g = sensf(time)*emoc*10. + (emoc*sensf(time)-2.0)*(price.flatten())
# ground truth
g

array([-534.5043093 , -166.94550133, -161.89743014, -426.86269013,
       -510.41272455, -186.83829862, -170.77927868, -121.46435273,
       -445.7142564 , -168.47584957, -114.79617859, -101.5171129 ,
       -587.72011145, -232.27548437, -232.82816252, -190.45861407,
       -119.57252921, -511.08323936, -417.74661819, -616.51951315])

$$ g = 10 \times s \phi_t  + (s \phi_t - 2) \times p$$

$$ y =  g + 100 + e $$

In [126]:
#y = (g - ymu)/ysd
#y

In [133]:

ypcor = 0.8
ynoise= 1
# e ~ N(\rho * v, 1 - \rho^2)
# ynoise = 1
# pnoise = 1
# (ypcor*ynoise/pnoise)*v, constant at every position
e = (ypcor*ynoise/pnoise)*v + rng.randn(n)*ynoise*np.sqrt(1-ypcor**2)
e.reshape(-1,1)

array([[ 1.73588365],
       [ 0.40175437],
       [-0.9170037 ],
       [-0.52197454],
       [-0.29581575],
       [ 2.12989921],
       [-0.02562201],
       [-0.88001376],
       [ 0.49038905],
       [ 1.82464636],
       [ 0.26452621],
       [ 0.44989487],
       [ 0.9363394 ],
       [-0.06010425],
       [ 0.22878067],
       [ 0.38715984],
       [ 0.22831449],
       [-0.50716445],
       [ 1.0325783 ],
       [ 0.99705099]])

In [94]:
g

array([-1845.19622053,  -607.50536832,  -599.20040052, -1429.91512951,
       -1718.34313502,  -673.47546777,  -606.49949637,  -454.64112043,
       -1538.33981677,  -625.21363327,  -432.5174214 ,  -387.28590188,
       -2025.51225814,  -837.35005822,  -833.88975455,  -691.73582852,
        -449.99896595, -1773.0758807 , -1464.60223059, -2101.93896313])

In [93]:
y = g + e + 100
y

array([-1744.24037929,  -506.49291377,  -499.6221461 , -1329.89052806,
       -1618.69876667,  -571.76535698,  -507.11579727,  -355.04908749,
       -1437.74275051,  -524.10564952,  -332.58102534,  -286.77257066,
       -1926.09599228,  -737.44276246,  -734.29607272,  -592.53101226,
        -349.95575375, -1673.39395219, -1363.95426117, -2002.16354487])

In [99]:
x, z, p, y ,g = demand(20, seed = 1, ynoise = 1., pnoise = 1., ypcor=0.8, use_images=False, test=False)

In [103]:
z

array([[ 0.04221375],
       [ 0.58281521],
       [-1.10061918],
       [ 1.14472371],
       [ 0.90159072],
       [ 0.50249434],
       [ 0.90085595],
       [-0.68372786],
       [-0.12289023],
       [-0.93576943],
       [-0.26788808],
       [ 0.53035547],
       [-0.69166075],
       [-0.39675353],
       [-0.6871727 ],
       [-0.84520564],
       [-0.67124613],
       [-0.0126646 ],
       [-1.11731035],
       [ 0.2344157 ]])

In [104]:
z = rng.randn(n) # z ~ n(0,1)
z

array([ 1.12141771,  0.40890054, -0.02461696, -0.77516162,  1.27375593,
        1.96710175, -1.85798186,  1.23616403,  1.62765075,  0.3380117 ,
       -1.19926803,  0.86334532, -0.1809203 , -0.60392063, -1.23005814,
        0.5505375 ,  0.79280687, -0.62353073,  0.52057634, -1.14434139])

In [13]:
def linear_data(n, 
                seed=None, 
                sig_d=0.5, 
                sig_y=2, 
                sig_t=1.5,
                alpha=4, 
                noiseless_t=False, 
                **kwargs):
    
    rng = np.random.RandomState(seed)
    nox = lambda z, d: z + 2*d
    house_price = lambda alpha, d, nox_val: alpha + 4*d + 2*nox_val

    d = rng.randn(n) * sig_d
    law = rng.randint(0, 2, n)

    if noiseless_t:
        t = nox(law, d.mean()) + sig_t*rng.randn(n)
    else:
        t = (nox(law, d) + sig_t*rng.randn(n) - 0.5) / 1.8
    z = law.reshape((-1, 1))
    x = np.zeros((n, 0))
    y = (house_price(alpha, d, t) + sig_y*rng.randn(n) - 5.)/5.
    g_true = lambda x, z, t: house_price(alpha, 0, t)
    return x, z, t.reshape((-1, 1)), y.reshape((-1, 1)), g_true


def main():
    pass


In [ ]:
def storeg(x, price):
    emoc = emocoef(x[:, 1:]) # s
    time = x[:, 0]           # t
    g = sensf(time)*emoc*10. + (emoc*sensf(time)-2.0)*(psd*price.flatten() + pmu)
    g_nrom = (g - ymu)/ysd
    return g_norm.reshape(-1, 1)


In [134]:
x, z, p, y ,g = demand(20, seed = 1, 
                       ynoise = 1., pnoise = 1., 
                       ypcor=0.8, 
                       use_images=False, test=False)